In [ ]:
## このプログラムは「CIFAR-10 - Object Recognition in Images」において
## 作成したノートブックで動作します
## ローカル環境のJupyter Notebookで作成したノートブックでも動作可能です

import numpy as np
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical

def prepare_data():
    """データを用意する
    
    Returns:
    X_train(ndarray):
        訓練データ(50000.32.32.3)
    X_test(ndarray):
        テストデータ(10000.32.32.3)
    y_train(ndarray):
        訓練データのOne-Hot化した正解ラベル(50000,10)
    y_train(ndarray):
        テストデータのOne-Hot化した正解ラベル10000,10)
    y_test_label(ndarray):
        テストデータの正解ラベル(10000)
    """
    (x_train, y_train), (x_test, y_test) = cifar10.load_data()
    
    # 訓練用とテスト用の画像データを正規化する
    x_train, x_test = x_train.astype('float32'), x_test.astype('float32')
    x_train, x_test = x_train/255.0, x_test/255.0
    
    # 訓練データとテストデータの正解ラベルを10クラスのOne-Hot表現に変換
    y_train, y_test = to_categorical(y_train), to_categorical(y_test)
    
    return x_train, x_test, y_train, y_test

In [ ]:
# モデルを生成する
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten # core layers
from tensorflow.keras.layers import Conv2D, MaxPooling2D    # convolution layers
from tensorflow.keras import optimizers

def make_convlayer():
    # Sequentialオブジェクトを生成
    model = Sequential()

    # 第1層: 畳み込み層1
    model.add(Conv2D(filters=64,
                     kernel_size=3,
                     padding='same', 
                     activation='relu',
                     input_shape=(32,32,3)))
    # 第2層: 2×2のプーリング層を配置
    model.add(MaxPooling2D(pool_size=2))

    # 第3層: 畳み込み層2
    model.add(Conv2D(filters = 128,
                     kernel_size = 3,
                     padding='same', 
                     activation='relu'))
    # 第4層: 2×2のプーリング層を配置
    model.add(MaxPooling2D(pool_size=2))

    # 第5層: 畳み込み層3
    model.add(Conv2D(filters=256,
                     kernel_size=3,
                     padding='same', 
                     activation='relu'))
    # 第6層: 2×2のプーリング層を配置
    model.add(MaxPooling2D(pool_size=2))

    # Flatten層
    model.add(Flatten())
    # ドロップアウトを設定
    model.add(Dropout(0.4))

    # 第7層
    model.add(Dense(512, activation='relu'))

    # 出力層
    model.add(Dense(10, activation = "softmax"))

    # モデルのコンパイル
    # オプティマイザーはAdam
    model.compile(loss="categorical_crossentropy",
                  optimizer=optimizers.Adam(lr=0.001),
                  metrics=["accuracy"])
    
    return model

In [ ]:
import math
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ReduceLROnPlateau

def train(x_train, x_test,
          y_train, y_test):
    
    # val_lossの改善が2エポック見られなかったら、学習率を0.5倍する。
    reduce_lr = ReduceLROnPlateau(
        monitor='val_accuracy', # 監視対象は検証データの精度
        factor=0.5,             # 学習率を減衰させる割合
        patience=5,             # 監視対象のエポック数
        verbose=1,              # 学習率を下げたときに通知する
        mode='max',             # 最高値を監視する
        min_lr=0.0001           # 学習率の下限
        )
    model = make_convlayer()
    callbacks_list = [reduce_lr]

    # データ拡張
    datagen = ImageDataGenerator(
        width_shift_range=0.1,  # 横サイズの0.1の割合でランダムに水平移動
        height_shift_range=0.1, # 縦サイズの0.1の割合でランダムに垂直移動
        rotation_range=10,      # 10度の範囲でランダムに回転させる
        zoom_range=0.1,         # ランダムに拡大
        horizontal_flip=True)   # 左右反転

    # ミニバッチのサイズ
    batch_size = 128
    # 学習回数
    epochs = 100

    # 学習を行う
    history = model.fit(
        # 拡張データをミニバッチの数だけ生成
        datagen.flow(x_train,
                     y_train,
                     batch_size=batch_size),
        # 1回の学習におけるステップ数
        # 画像の枚数をミニバッチのサイズで割った整数値
        steps_per_epoch=x_train.shape[0] // batch_size,
        epochs=epochs, # 学習回数
        verbose=1,     # 学習の進捗状況を出力する
        # テストデータ
        validation_data=(x_test, y_test),
        callbacks=callbacks_list
    )
    
    return history

In [ ]:
x_train, x_test, y_train, y_test = prepare_data()

In [ ]:
%%time
history = train(x_train, x_test, y_train, y_test)

In [ ]:
# 損失と正解率（精度）の推移をグラフにする

%matplotlib inline
import matplotlib.pyplot as plt

# プロット図のサイズを設定
plt.ﬁgure(ﬁgsize=(15, 10))
# プロット図を縮小して図の間のスペースを空ける
plt.subplots_adjust(wspace=0.2)

# 2×1のグリッドの上部にプロット
plt.subplot(2, 1, 1)
# 訓練データの精度をプロット
plt.plot(
    history.history['accuracy'], label='train', color='black')
# テストデータの精度をプロット
plt.plot(
    history.history['val_accuracy'], label='Val Acc',color='red')
plt.legend()         # 凡例を表示
plt.grid()           # グリッド表示
plt.xlabel('Epoch')  # x軸ラベル
plt.ylabel('Acc')    # y軸ラベル

# 2×1のグリッドの下部にプロット
plt.subplot(2, 1, 2)
# 学習率をプロット
plt.plot(history.history['lr'],
         label='Learning Rate',
         color='blue')
plt.legend()         # 凡例を表示
plt.grid()           # グリッド表示
plt.xlabel('Epoch')  # x軸ラベル
plt.ylabel('Learning Rate')    # y軸ラベル
plt.show()